In [1]:
import os
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Directorios de datos
train_dir = '../data/train' 
valid_dir = '../data/valid'
test_dir = '../data/test'

2024-08-14 22:35:22.853210: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
# Generador de datos con aumentación para el entrenamiento y normalización para validación y prueba
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

valid_test_datagen = ImageDataGenerator(rescale=1./255)

# Crear los generadores
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=128,
    class_mode='categorical'
)

validation_generator = valid_test_datagen.flow_from_directory(
    valid_dir,
    target_size=(224, 224),
    batch_size=128,
    class_mode='categorical'
)

test_generator = valid_test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=128,
    class_mode='categorical',
    shuffle=False
)

# Cargar el modelo base preentrenado EfficientNetB0
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Congelar las capas del modelo base
base_model.trainable = False

# Añadir capas superiores personalizadas
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(2, activation='softmax')(x)  # Suponiendo 2 clases: Melanoma y NotMelanoma

# Crear el modelo completo
model = models.Model(inputs=base_model.input, outputs=outputs)

# Compilar el modelo
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Mostrar la arquitectura del modelo
model.summary()

# Configurar callbacks
checkpoint = ModelCheckpoint('../models/best_melanomaornot_model.keras',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, min_lr=1e-7)

In [ ]:
# Entrenar el modelo
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=30,
    callbacks=[checkpoint, early_stopping, reduce_lr]
)

In [ ]:
# EVALUACION

# Cargar el mejor modelo guardado
model = models.load_model('../models/best_melanomaornot_model.keras')

# Evaluar el modelo en el conjunto de prueba
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

# Obtener predicciones
predictions = model.predict(test_generator, steps=test_generator.samples // test_generator.batch_size)
predicted_classes = np.argmax(predictions, axis=1)

In [ ]:
# Generar reporte de clasificación
true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

report = classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report)

# Mostrar la matriz de confusión
conf_matrix = confusion_matrix(true_classes, predicted_classes)
print("Confusion Matrix")
print(conf_matrix)

In [ ]:
# Opcional: Ajuste Fino del modelo
# Descongelar las últimas 20 capas del modelo base
base_model.trainable = True
for layer in base_model.layers[:-20]:
    layer.trainable = False

# Compilar y entrenar nuevamente
model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history_fine = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10,
    callbacks=[checkpoint, early_stopping, reduce_lr]
)

In [ ]:
# Re-evaluación en el conjunto de prueba después del ajuste fino
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f'Fine-tuned Test Loss: {test_loss}')
print(f'Fine-tuned Test Accuracy: {test_accuracy}')

In [ ]:
# Obtener predicciones después del ajuste fino
predictions = model.predict(test_generator, steps=test_generator.samples // test_generator.batch_size)
predicted_classes = np.argmax(predictions, axis=1)

In [ ]:
# Generar reporte de clasificación después del ajuste fino
report = classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report)

In [ ]:
# Mostrar la matriz de confusión después del ajuste fino
conf_matrix = confusion_matrix(true_classes, predicted_classes)
print("Confusion Matrix after Fine-tuning")
print(conf_matrix)